### XML Parser For Stats Functions  
### Sean Warlick  
### July 23, 2016

This note book is used to develop the code needed to parse the XML returned from the `TruliaStats` library.  We interactively explore the XML to find to aquire the desired data.  A notebook was used so we did not need to continually call the API.  The bulk of the code here will be discarded. 



In [ ]:
# This is needed to accomidate the new file structure designed on 8/2
import sys
sys.path.append('/Users/SeanWarlick/Documents/GitHub/Real_Estate_Data/python')

In [ ]:
# Import Packages
import api_function as api
import xml.etree.ElementTree as et 

In [ ]:
state_data = api.state_stats("", "WA", '2016-01-01', '2016-01-09', #API KEY HERE)

In [ ]:
print type(state_data)

state_data2 = et.fromstring(state_data) # Create Element from string
state_tree = et.ElementTree(state_data2) # Convert Element to Element Tree

In [ ]:
#print state_data

### Explore Structure in Response Tags
TruliaStats is child of interest in the response tag. 

In [ ]:
# Find Children in Location Stats
state_response = state_tree.find(".//TruliaStats")

for child in state_response:
    print child.tag, child.attrib, child.text

There are three nodes under *TruliaStats*.  We will explore these sequentially, starting with trafficStats. 

In [ ]:
traffic = state_tree.find('.//trafficStats')

In [ ]:
for child in traffic:
    print child.tag, child.attrib, child.text

Beneith the *trafficStats* node we have a group of nodes named *trafficStat*.  Now we look at one of these nodes to explore how the data is stored. 

In [ ]:
traffic_data = traffic.find("trafficStat")

In [ ]:
for child in traffic_data:
    print child.tag, child.attrib, child.text

This is as far down as we need to go on the state traffic data.  We can now develop code to parse all the returned records!.  

In [ ]:
# KEEP THIS CODE
traffic_data_collect = state_tree.findall(".//trafficStat")

storage = {"date":[], "value":[]} # Create storage variable

for i in traffic_data_collect:
    date = i.find("date").text
    storage["date"].append(date)
    
    val = i.find("percentNationalTraffic").text
    storage["value"].append(val)


In [ ]:
print storage

Now that we have parsed the trafic data we need to explore the listing data. 
We'll start by finding the children under listing data. 

In [ ]:
listingstats = state_tree.find(".//listingStats")
for child in listingstats:
    print child.tag, child.attrib, child.text

In [ ]:
listing = state_tree.find(".//listingStat")
for child in listing:
    print child.tag, child.attrib, child.text

In [ ]:
listingPrice = state_tree.find(".//listingPrice")
for child in listingPrice:
    print child.tag, child.attrib, child.text

In [ ]:
subcat = state_tree.find(".//subcategory")
for child in subcat:
    print child.tag, child.attrib, child.text

The data of interest is with in the subcategory tags.  We also need to keep the text from the weekEndingDate tag.  
To correctly parse this we will need to go back up to the listing stat tag.  From there we will obtain the end date which we will use with each listing category

In [ ]:
listing_data = state_tree.findall(".//listingStat")

storage = {"date":[], "type":[], "properties":[], "medianPrice":[], "avgPrice":[]}

for records in listing_data:
    record_date = records.find("weekEndingDate").text # Extract record date
    #print record_date
    categories = records.findall(".//subcategory") # Find all subcategory for the given weekend date
    
    for category in categories:
        storage["date"].append(record_date) 

        tp = category.find("type").text
        storage["type"].append(tp)
        
        prop = category.find("numberOfProperties").text
        storage["properties"].append(prop)
        
        median = category.find("medianListingPrice").text
        storage["medianPrice"].append(median)
        
        avg = category.find("averageListingPrice").text
        storage["avgPrice"].append(avg)
        
    
    

In [ ]:
print storage

Now we just need to turn this dictonary into an array. 

### Parsing `getCityStats`

In [ ]:
x = api.city_stats("Seattle", "WA", '2016-01-01', '2016-01-09', #API KEY HERE)

In [ ]:
city_data = et.fromstring(x) # Create Element from string
city_tree = et.ElementTree(city_data) # Convert Element to Element Tree

In [ ]:
# KEEP THIS CODE 
listing_data = city_tree.findall(".//listingStat")

storage = {"date":[], "type":[], "properties":[], "medianPrice":[], "avgPrice":[]}
for records in listing_data:
    record_date = records.find("weekEndingDate").text # Extract record date
    #print record_date
    categories = records.findall(".//subcategory") # Find all subcategory for the given weekend date
    
    for category in categories:
        storage["date"].append(record_date) 

        tp = category.find("type").text
        storage["type"].append(tp)
        
        prop = category.find("numberOfProperties").text
        storage["properties"].append(prop)
        
        median = category.find("medianListingPrice").text
        storage["medianPrice"].append(median)
        
        avg = category.find("averageListingPrice").text
        storage["avgPrice"].append(avg)

In [ ]:
print storage

### Parse listing data from `getZipCodeStats`

In [ ]:
zip_string = api.zip_stats('20002', "2016-01-01", "2016-01-09", #API KEY HERE)

In [ ]:
zip_data = et.fromstring(zip_string) # Create Element from string
zip_tree = et.ElementTree(zip_data) # Convert Element to Element Tree

In [ ]:
zip_listing = zip_tree.find(".//listingStat")

In [ ]:
for child in zip_listing:
    print child.tag, child.attrib, child.text

In [ ]:
listing = zip_listing.find("listingPrice")
for child in listing:
    print child.tag, child.attrib, child.text

In [ ]:
cat = listing.find("subcategory")

for child in cat:
    print child.tag, child.attrib, child.text

In [ ]:
listing_data = zip_tree.findall(".//listingStat")

storage = {"date":[], "type":[], "properties":[], "medianPrice":[], "avgPrice":[]}

for records in listing_data:
    record_date = records.find("weekEndingDate").text
    
    categories = records.findall(".//subcategory")
    for category in categories:
        storage["date"].append(record_date) 

        tp = category.find("type").text
        storage["type"].append(tp)
        
        prop = category.find("numberOfProperties").text
        storage["properties"].append(prop)
        
        median = category.find("medianListingPrice").text
        storage["medianPrice"].append(median)
        
        avg = category.find("averageListingPrice").text
        storage["avgPrice"].append(avg)

In [ ]:
print storage["date"], storage["medianPrice"]